In [2]:
import pandas as pd
import re
import numpy as np
from collections import Counter
import unicodedata

In [3]:
df = pd.read_csv("./5_crawl_tags_and_clean_null_description.csv")

In [4]:
df.head()

,ID,Description,depth,story_tags_list,cover_img,title,Lượt đọc,Lượt bình chọn,Chương,recommended_stories,badges,Thời gian
0,0,Cuộc sống và chuyện tình cảm của 12 chòm sao t...,0,"['12chomsao', '12cs', '12cunghoàngđạo', 'bíẩn'...",https://img.wattpad.com/cover/51678600-512-k93...,Vampire và người sói (12 chòm sao)[full],69463.0,3355.0,32.0,"['0020', '0021', '0022', '0023', '0024', '0025...",NaN,NaN
1,1,"Thế kỷ 21, nàng là một trong những người nổi t...",0,"['bíẩn', 'cungđấu', 'dieutra', 'thriller', 'tr...",https://img.wattpad.com/cover/128268448-512-k3...,Nữ Ngỗ Tác Họa Cốt,3529078.0,128419.0,160.0,"['0030', '0031', '0032', '0033', '0034', '0035...",['Đang sáng tác'],NaN
2,2,Đây là những gì mình sưu tầm được. Vì khá thíc...,0,"['bíẩn', 'cryptic', 'fbi']",https://img.wattpad.com/cover/68364418-512-k14...,CRYPTIC ( Câu đố Cryptic Ft. Đáp án),87261.0,2780.0,13.0,"['0037', '0036', '0035', '0030', '0034', '0032...",['Đang sáng tác'],NaN
3,3,Tình yêu của một tên giết người dưới góc nhìn ...,0,"['blood', 'bíẩn', 'horror', 'killer', 'kinhdi'...",https://img.wattpad.com/cover/89250906-512-k22...,Em sẽ là nạn nhân của anh,34795.0,2049.0,17.0,"['0032', '0036', '0031', '0034', '0035', '0038...",['Hoàn thành'],NaN
4,4,Hán Việt: Tuyệt vọng giáo thất\n \n Tác ...,0,"['bíẩn', 'edit', 'hiệnđại', 'hồihộp', 'khoahọc...",https://img.wattpad.com/cover/182991023-256-k8...,LỚP HỌC RÙNG RỢN,25746.0,1068.0,NaN,"['0036', '0035', '0038', '0032', '0039', '0030...",['Đang sáng tác'],NaN


In [5]:
regex = re.compile('[^\u0000-\u007F\u0080-\u00FF\u0100-\u024F\u1E00-\u1EFF\u0300-\u036F]')

def clean_up(text):
    if type(text) == type(np.nan):
        return text
    
    # Clean out non-Latin characters
    text = regex.sub('', text)
    
    # Clean out - symbol
    text = re.sub(r'-', '', text)
    
    return text

df['story_tags_list'] = df['story_tags_list'].apply(clean_up)

In [6]:
def remove_vietnamese_diacritics(text):
    if type(text) == type(np.nan):
        return text
    
    text = text.replace('đ', 'd')
    # Normalize the text to decomposed Unicode form
    normalized_text = unicodedata.normalize('NFD', text)
    # Use regex to remove combining diacritical marks
    cleaned_text = re.sub('[\u0300-\u036f]', '', normalized_text)
    return cleaned_text


df['story_tags_list'] = df['story_tags_list'].apply(remove_vietnamese_diacritics)

In [7]:
def str_to_list(string):
    if isinstance(string, list):
        return string
    return string[2:-2].split("', '")

In [8]:
tags_list = []

for i in range(len(df)):
    tags = df.iloc[i]["story_tags_list"]
        
    new_tags = []
    delete_tags = False
    
    if type(tags) == type(np.nan):
        print(tags)
        continue
        
    df.at[i, "story_tags_list"] = str_to_list(tags)
    
    for tag in str_to_list(tags):
        if len(tag) == 0:
            delete_tags = True
            continue
        
        new_tags.append(tag)
    
    if delete_tags:
        df.at[i, "story_tags_list"] = new_tags
    
    tags_list.extend(new_tags)

In [9]:
Counter(tags_list)

Counter({'romance': 546,
         'love': 442,
         'fanfiction': 279,
         'hiendai': 275,
         'dammy': 247,
         'ngontinh': 241,
         'langman': 221,
         'bl': 171,
         'sung': 155,
         'drama': 144,
         'texttospeech': 142,
         'ngot': 133,
         'bts': 130,
         'codai': 129,
         'fanfic': 124,
         'teenfiction': 123,
         'haihuoc': 118,
         'fantasy': 117,
         'friendship': 112,
         'tinhcam': 110,
         'lovestory': 110,
         'humor': 110,
         'badboy': 109,
         'kinhdi': 107,
         'family': 106,
         'hocduong': 101,
         'teen': 99,
         'jungkook': 94,
         'highschool': 94,
         'fiction': 92,
         'wattpadoriginals': 92,
         'youngadult': 92,
         'danmei': 90,
         '1v1': 89,
         'hanhdong': 86,
         '12chomsao': 85,
         'nguoc': 85,
         '1x1': 83,
         'trongsinh': 83,
         'he': 82,
         'boylove': 81,

In [10]:
df.to_csv("./6_filter_latin_tags.csv", index=False)

In [11]:
len(tags_list)

34879

In [12]:
len(Counter(tags_list).keys())

10786

In [13]:
for i in range(len(df)):
    tags = df.at[i, "story_tags_list"]
    
    if type(tags) == type(np.nan):
        print(f"0: {i}")
        continue
        
    #tags = str_to_list(tags)
        
    if len(tags) == 0:
        print(f"1: {i}")
        print(tags)
        continue
        
    if len(tags) == 1 and len(tags[0]) == 1:
        print(f"2: {i}")
        continue

1: 398
[]
1: 1560
[]


In [14]:
deleted = [398, 1560]

In [17]:
df = df.loc[~df.index.isin(deleted)]

In [18]:
df.to_csv("./6_filter_latin_tags.csv", index=False)